In [1]:
import platform
import folium
import math
from tqdm import tqdm

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action = 'ignore')

from geoband.API import *
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

if platform.system() == 'Darwin': # 맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': # 윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': # 리눅스 (구글 Colab)
        plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## 현재까지 구축했던 데이터 불러오기

In [2]:
accident_count_filter_24 = pd.read_csv('accident_count_filter_24.csv', index_col = 0)
accident_count_filter_24.head()

,X,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,...,차대사람 -30대,차대사람 -40대,차대사람 -50대,차대사람 -60대 이상,차대차 -20대,차대차 -20대 미만,차대차 -30대,차대차 -40대,차대차 -50대,차대차 -60대 이상
1,0,다바931203,2,MULTIPOLYGON (((127.4230710131166 36.380134552...,2,2,127.423628,36.380586,1,3,...,0,0,0,1,0,0,0,1,0,0.0
2,1,다바861174,0,MULTIPOLYGON (((127.3450791441312 36.353914265...,0,0,127.345636,36.354366,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,2,다바900127,1,MULTIPOLYGON (((127.3886063974092 36.311590216...,1,1,127.389163,36.312042,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,3,다바885170,15,MULTIPOLYGON (((127.3718339133395 36.350339794...,15,15,127.372390,36.350791,0,0,...,1,0,1,0,1,0,5,3,2,1.0
5,4,다바957151,4,MULTIPOLYGON (((127.452087475365 36.3332696085...,4,4,127.452644,36.333721,2,3,...,1,0,0,0,1,0,1,0,0,0.0


## 24. 교통사고내역 데이터로 사망자, 중상자, 경상자의 수를 gid 격자에 따라 현재까지 구축한 데이터와 병합

In [3]:
accident_list = pd.read_csv('1.대전광역시_교통사고내역(2017~2019).csv')
count = accident_list.groupby(['gid', '사망자수', '중상자수', '경상자수']).count()
count.reset_index(inplace=True)
count = count[['gid', '사망자수', '중상자수', '경상자수']]
count.head()

,gid,사망자수,중상자수,경상자수
0,다바780093,0,0,1
1,다바781090,0,0,2
2,다바781091,0,1,0
3,다바781091,0,2,1
4,다바787132,0,1,0


In [4]:
count_death = count[['gid','사망자수']].groupby('gid').sum('사망자수')
count_inj = count[['gid','중상자수']].groupby('gid').sum('중상자수')
count_weak = count[['gid','경상자수']].groupby('gid').sum('경상자수')

In [5]:
count_all = count_death.merge(count_weak, on='gid', how='left').merge(count_inj, on='gid', how='left')
count_all.reset_index(inplace=True)

count_all 

,gid,사망자수,경상자수,중상자수
0,다바780093,0,1,0
1,다바781090,0,2,0
2,다바781091,0,1,3
3,다바787132,0,0,1
4,다바787134,0,1,2
...,...,...,...,...
6063,라바010169,1,6,0
6064,라바016173,0,0,1
6065,라바024244,1,0,0
6066,라바033232,0,2,0


In [6]:
accident_count_filter_25 = accident_count_filter_24.merge(count_all, on='gid', how='left')
accident_count_filter_25.head()

,X,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,...,차대사람 -60대 이상,차대차 -20대,차대차 -20대 미만,차대차 -30대,차대차 -40대,차대차 -50대,차대차 -60대 이상,사망자수,경상자수,중상자수
0,0,다바931203,2,MULTIPOLYGON (((127.4230710131166 36.380134552...,2,2,127.423628,36.380586,1,3,...,1,0,0,0,1,0,0.0,0.0,1.0,1.0
1,1,다바861174,0,MULTIPOLYGON (((127.3450791441312 36.353914265...,0,0,127.345636,36.354366,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,2,다바900127,1,MULTIPOLYGON (((127.3886063974092 36.311590216...,1,1,127.389163,36.312042,0,0,...,0,0,0,0,0,0,NaN,0.0,3.0,0.0
3,3,다바885170,15,MULTIPOLYGON (((127.3718339133395 36.350339794...,15,15,127.372390,36.350791,0,0,...,0,1,0,5,3,2,1.0,0.0,9.0,4.0
4,4,다바957151,4,MULTIPOLYGON (((127.452087475365 36.3332696085...,4,4,127.452644,36.333721,2,3,...,0,1,0,1,0,0,0.0,0.0,3.0,0.0


In [7]:
accident_count_filter_25 = accident_count_filter_25.fillna(0)
accident_count_filter_25 = accident_count_filter_25.iloc[:, 1:]
print(np.shape(accident_count_filter_25))
accident_count_filter_25.head(3)

(5556, 48)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,...,차대사람 -60대 이상,차대차 -20대,차대차 -20대 미만,차대차 -30대,차대차 -40대,차대차 -50대,차대차 -60대 이상,사망자수,경상자수,중상자수
0,다바931203,2,MULTIPOLYGON (((127.4230710131166 36.380134552...,2,2,127.423628,36.380586,1,3,0,...,1,0,0,0,1,0,0.0,0.0,1.0,1.0
1,다바861174,0,MULTIPOLYGON (((127.3450791441312 36.353914265...,0,0,127.345636,36.354366,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,다바900127,1,MULTIPOLYGON (((127.3886063974092 36.311590216...,1,1,127.389163,36.312042,0,0,0,...,0,0,0,0,0,0,0.0,0.0,3.0,0.0


In [8]:
accident_count_filter_25.to_csv('accident_count_filter_25.csv')
# 신호등(보행등), 신호등(차량등), cctv수, 혼잡빈도, 혼잡시간, 교통추정량, 날씨, 안전지대수, 중앙분리대수, 정차금지지대수, 도로속도표시수
# 교통안전표지수, 노드개수, 횡단보도수, 건물면적, 자동차대수, 총거주인구수, 연령 및 사고유형, 사망&경상&중상자수에 따라 나눈 데이터까지 merge한 상태.